<a href="https://colab.research.google.com/github/mdjamina/m1_ml_lang_detector/blob/main/data_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus
[Tatoeba](https://tatoeba.org/fr/downloads
) est une collection de phrases et de traductions.

## Téléchargement du corpus

In [ ]:
!wget https://iso639-3.sil.org/sites/iso639-3/files/downloads/iso-639-3-macrolanguages.tab

In [ ]:
!wget https://downloads.tatoeba.org/exports/sentences.tar.bz2

Extraction de l'archive téléchargée 

## Chargement du corpus

In [7]:
import pandas as pd

data = pd.read_csv('sentences.tar.bz2', sep='\t', header=None,compression='bz2')
data.columns = ['id','I_Id', 'content']


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
data.shape

(10262675, 3)

## Nettoyage des données

In [9]:
#suppression des valeurs NAN
data.dropna(subset = ["I_Id"], inplace=True)
#identification des colonnes qui contiennent des valeurs NAN (non renseignées)
data.isnull().any()

id         False
I_Id       False
content    False
dtype: bool

## regrouppement des dialects par langue principale

In [10]:
langs ={'eng':'English','pol':'Polish','deu':'German','fra':'French','spa':'Spanish','ita':'Italian'
,'tur':'Turkish','por':'Portuguese','rus':'Russian','ukr':'Ukrainian','nld':'Dutch','bul':'Bulgarian'
,'ell':'Greek','swe':'Swedish','hun':'Hungarian','gle':'Irish','lav':'Latvian','dan':'Danish','fin':'Finnish',
'ara':'Arabic','zho':'Chinese','hin':'Hindi','jpn':'Japanese','fas':'Persian','kor':'Korean','hye':'Armenian','swa':'Swahili','ber':'Berber','kab':'Kabyle'}

In [ ]:
#TODO
# site iso langues avec leur regroupement
# telecharger fichier csv(iso) contient : langue : arb=dz, ara=egy ..,  le telecharger avec  pandas data frames 

In [ ]:
import pandas as pd
macro_lang = pd.read_csv('https://raw.githubusercontent.com/mdjamina/m1_ml_lang_detector/main/data/iso-639-3-macrolanguages.tab', sep='\t')
macro_lang.columns = ['lang',	'I_Id',	'I_Status']

macro_lang

In [35]:
#jointure entre la table des sentences et la table macrolanguages
df = pd.merge(data,macro_lang, how='left',on='I_Id')
df.lang.fillna(df.I_Id, inplace=True)

#suppression des columns 'I_Status', 'id' et 'I_Id'
df = df.drop(columns=['I_Status', 'id','I_Id'])

#df[~df['lang'].isin(macro_lang['lang'])]

#garder que les langues selectionnées dans la liste langs
df=df[df['lang'].isin(langs.keys())]

df.shape


(8441248, 2)

In [63]:
df_counts = df.groupby(['lang']).count()

#df_counts[(df_counts['content']>50000) ]
 
to_reduce0 = df[df['lang'].isin(df_counts[(df_counts['content']>=100000) & (df_counts['content']<500000) ].index)]
to_reduce1 = df[df['lang'].isin(df_counts[(df_counts['content']>=500000) & (df_counts['content']<1000000) ].index)]
to_reduce2 = df[df['lang'].isin(df_counts[(df_counts['content']>=1000000 ) ].index)]

df = df[df['lang'].isin(df_counts[(df_counts['content']<100000) ].index)]


In [64]:
df.groupby(['lang']).count()

,content
lang,
ara,42128
bul,24521
dan,53561
ell,34791
fas,24783
gle,2645
hin,14559
hye,2249
kor,8947


In [65]:
to_reduce0.groupby(['lang']).count()

,content
lang,
fin,138867
fra,497104
hun,353702
jpn,218894
nld,154128
pol,115926
por,394187
spa,367512
ukr,177839


In [66]:
to_reduce1.groupby(['lang']).count()

,content
lang,
ber,589554
deu,581093
ita,802842
kab,513886
rus,903431
tur,717173


In [40]:
to_reduce2.groupby(['lang']).count()

,content
lang,
eng,1572075


In [46]:
from sklearn.model_selection import train_test_split 
#on prends que "




In [67]:
content_reduced0, _, lang_reduced0, _ = train_test_split(to_reduce0['content'], to_reduce0['lang'], test_size=0.60, random_state=42) 
reduced0 = pd.DataFrame({'content':content_reduced0,'lang':lang_reduced0})


reduced0.groupby(['lang']).count()

,content
lang,
fin,55534
fra,199418
hun,141292
jpn,87774
nld,61399
pol,46410
por,157939
spa,146608
ukr,70889


In [68]:
content_reduced1, _, lang_reduced1, _ = train_test_split(to_reduce1['content'], to_reduce1['lang'], test_size=0.80, random_state=42) 

reduced1 = pd.DataFrame({'content':content_reduced1,'lang':lang_reduced1})


reduced1.groupby(['lang']).count()

,content
lang,
ber,118117
deu,116324
ita,160661
kab,102322
rus,180735
tur,143436


In [69]:
content_reduced2, _, lang_reduced2, _ = train_test_split(to_reduce2['content'], to_reduce2['lang'], test_size=0.92, random_state=42) 
reduced2 = pd.DataFrame({'content':content_reduced2,'lang':lang_reduced2})


reduced2.groupby(['lang']).count()

,content
lang,
eng,125766


In [70]:

df = df.append(reduced0)
df = df.append(reduced1)
df = df.append(reduced2)
df.groupby(['lang']).count()

,content
lang,
ara,42128
ber,118117
bul,24521
dan,53561
deu,116324
ell,34791
eng,125766
fas,24783
fin,55534


In [72]:
df.to_pickle('df_corpus.pkl')